# 🔍 Pesquisa - Básico",

Este notebook tem como objetivo apresentar **consultas básicas no Elasticsearch**.

Vamos explorar como realizar pesquisas simples utilizando a API de busca da ferramenta, com exemplos práticos e objetivos.

A ideia é mostrar como localizar documentos com base em seus campos, utilizando filtros e parâmetros comuns do dia a dia.

Para maiores, informações, acessar o [link](https://www.elastic.co/docs/solutions/search/full-text)

Para ilustrar como fazer pesquisa, importaremos os dados de algumas músicas extraidos do site [vagalume](https://www.vagalume.com.br/).

Criando o conector e o indice para salvar as músicas

In [1]:
import json
from pathlib import Path

In [2]:
#Carregando as músicas de diferentes artistas a partir de arquivos JSON
emicidade_musicas = None
emicidade_path = Path('emicida.json')
with emicidade_path.open('r', encoding='utf-8') as f:
    emicidade_musicas = json.load(f)

charlie_musicas = None
charlie_path = Path('charlie.json')
with charlie_path.open('r', encoding='utf-8') as f:
    charlie_musicas = json.load(f)

Cada objeto dentro da lista possui as seguintes chaves

letra: Letra da música

titulo: Nome da música

Autor: Autores da música

Compositor: Compositores da música

Editor: Editores da música

URL: link dentro do site vagalume

Administração: Administração da música

Ano: Ano da publicação

Extracao: Dia que os dados foram extraidos

In [3]:
emicidade_musicas[0]

{'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
 'autor': 'Emicida',
 'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angústia é eu ter calma e a vida escada  \n  Junto ler almas pra além da pressão  \n  As voz em declive na mão desse Barrabás  \n  Onde o milagre jaz  \n  Só prova a urgência de livros  \n  perante o estrago que um sábio faz  \n  O mestre em dívidas avidas  \n  Sem noção do que são dádivas  \n  No tempo onde a única que corre livre  \n  aqui sã

In [ ]:
# Criando o conector com elasticsearch
# Modificar a variável PASSWORD para a senha configurado no .env
from elasticsearch import Elasticsearch

HOST = 'http://localhost:9200'
USERNAME = 'elastic'
PASSWORD = 'test123'

client = Elasticsearch(
    HOST,
    basic_auth=(USERNAME,PASSWORD)
)


In [11]:
#Verificando a conexão com o Elasticsearch
client.info()

ObjectApiResponse({'name': '73348a783ff1', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'dTLY6PORQHSoQ3_mIDx4VA', 'version': {'number': '8.5.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '4ed5ee9afac63de92ec98f404ccbed7d3ba9584e', 'build_date': '2022-12-05T18:22:22.226119656Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [14]:
#Criando o índice para as músicas
INDEX_MUSICAS = 'musicas_pesquisa'

if client.indices.exists(index=INDEX_MUSICAS):
    client.indices.delete(index=INDEX_MUSICAS)
client.indices.create(index=INDEX_MUSICAS)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'musicas_pesquisa'})

In [15]:
#Salvando as músicas do Emicida no Elasticsearch
for musica in emicidade_musicas:
    client.index(
        index=INDEX_MUSICAS,
        body=musica
    )

for musica in charlie_musicas:
    client.index(
        index=INDEX_MUSICAS,
        body=musica
    )

### Pesquisa básica de texto

Inicialmente, faremos algumas pesquisas básicas e entender como elastisearch retorna 
A linguagem de pesquisa chama DSL, segue o [link](https://www.elastic.co/docs/reference/query-languages/)


In [16]:
#Pesquisando pela a palavra "Emicida" no campo "autor"

response = client.search(
    index=INDEX_MUSICAS,
    query={
        "match":{
            "autor":"Emicida"
        }
    }
)

In [18]:
response.body

{'took': 69,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 25, 'relation': 'eq'},
  'max_score': 0.8713851,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'l35WN5cBiMBQPnh-BvRO',
    '_score': 0.8713851,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
     'autor': 'Emicida',
     'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angú

O Elasticsearch retorna os seguintes campos

1. 'took':  Quanto tempo para fazer a pesquisa (em milesegundos)
2. 'timed_out': Se teve timeout
3. '_shards': Se teve sucesso (1 = True) ou não
4. 'hits': Um objeto com as seguintes chaves

    4.1 total: Quantos objetos satisfazem a pesquisa (neste caso, são 25 objetos)
    
    4.2' max_score': Qual é a maior pontuação. O elastisearch informa uma pontuação para indicar a relevância do objeto perante a pesquisa. Quanto maior a pontuação, mais relevante é o documento. Para verificar o cálculo, acesse o [link](https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables)

    4.3 hits: Lista dos 10 primeiros objetos e suas pontuações e id (código único) 


In [19]:
#Pesquisando pela a palavra "Manuel" no campo "autor". Não existe música com esse autor
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "match":{
            "autor":"Manuel"
        }
    }
)

In [21]:
response.body

{'took': 9,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

### Should, Must e Filter


A linguagem DSL possui 3 componentes/palavras chaves importantes: Should,Must e Filter. Cada um destes desempenha um papel fundamental para pesquisar e alterar a pontuação da pesquisa

In [ ]:
# O termo Must indica que todos os critérios devem ser atendidos
# Pesquisando pela a palavra Emicida na chave "autor"  e a palavra "Amor" na chave  "letra"
# Podemos 
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "bool":{
            "must":[
                {
                    "match":{
                        "autor":"Emicida"
                    }
                },
                {
                    "match":{
                        "letra":"Amor"
                    }
                }
            ]
        }
    }
)



In [24]:
response.body

{'took': 42,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 7, 'relation': 'eq'},
  'max_score': 2.7495031,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'l35WN5cBiMBQPnh-BvRO',
    '_score': 2.7495031,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
     'autor': 'Emicida',
     'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angús

In [25]:
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "bool":{
            "must":[
                {
                    "match":{
                        "autor":"Charlie"
                    }
                },
                {
                    "match":{
                        "letra":"Amor"
                    }
                }
            ]
        }
    }
)

In [26]:
response.body

{'took': 8,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10, 'relation': 'eq'},
  'max_score': 2.5110602,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'sH5WN5cBiMBQPnh-CfSf',
    '_score': 2.5110602,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Meu Novo Mundo',
     'autor': 'Charlie Brown Jr',
     'letra': 'Como se o silêncio dissesse tudo  \n  Um sentimento bom que me leva pra outro mundo  \n  A vontade de te ver já é maior que tudo  \n  Não existem distâncias no meu novo mundo  \n    \n  Tipo coisas da sétima arte  \n  Aconteceu sem que eu imaginasse  \n    \n  Sonho de consumo cantar na sua festa  \n  Vem dançar comigo  \n  Aproveita e me sequestra  \n  Amor vagabundo, intenso ou muita pressa  \n  Não sei como termina mas sei como começa  \n    \n  Fiz essa canção pra dizer algumas coisas  \n  Cuidado com o destino  \n  Ele brinca com as pessoas  \n  Tipo uma foto com sorriso inocen

In [27]:
# O termo Should indica que pelo menos um dos critérios deve ser atendido. Isso altera a relevância do objeto
response = client.search(
    index=INDEX_MUSICAS,
    query={
        "bool":{
            "should":[
                {
                    "match":{
                        "autor":"Emicida"
                    }
                }
            ]
        }
    }
)

In [28]:
response.body

{'took': 26,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 25, 'relation': 'eq'},
  'max_score': 0.8713851,
  'hits': [{'_index': 'musicas_pesquisa',
    '_id': 'l35WN5cBiMBQPnh-BvRO',
    '_score': 0.8713851,
    '_ignored': ['letra.keyword'],
    '_source': {'titulo': 'Principia (Part. Pastor Henrique Vieira, Fabiana Cozza & Pastoras do Rosário)',
     'autor': 'Emicida',
     'letra': "Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n  Lá-ia, lá-ia, lá-ia  \n    \n  O cheiro doce da arruda  \n  Penso em Buda calmo  \n  Tenso eu busco uma ajuda às vezes me vem o Salmo  \n  Tira a visão que iluda, é tipo um oftalmo'  \n  E eu, que vejo além de um palmo  \n  Por mim, tô Ubuntu, ó, uau  \n  Se for pra crer num terreno  \n  Só no que nóis tá vendo memo'  \n  Resumo do plano é baixo, pequeno e mundano  \n  Sujo, inferno e veneno  \n  Frio, inverno e sereno  \n  Repressão e regressão  \n  Angú

Observe que retornou 

### Outras pesquisas